<div class="alert alert-info">
Ссылка для просмотра ноутбука в интерактивном режиме для использования гиперссылок и корректного отображения разметки:<br>
<a href='https://nbviewer.org/github/yulianikola/portfolio/blob/master/data_analysis_python/0_syntax/pandas.ipynb'>pandas</a></div>

### Синтаксис Pandas

<p id="0">
<ul type="square"><a href="#1"><li>Задание 1 (value_counts, groupby, sort_values, loc, merge)</li></a>
<a href="#2"><li>Задание 2 (unique, str.contains, between, isin)</li></a>
<a href="#3"><li>Задание 3 (read_html)</li></a>
<a href="#4"><li>Задание 4 (loc, apply)</li></a>
<a href="#5"><li>Задание 5 (str.contains, regex)</li></a>
<a href="#6"><li>Задание 6 (groupby, agg)</li></a>
<a href="#7"><li>Задание 7 (nunique, duplicated, merge)</li></a>

In [1]:
import pandas as pd

<p id="1">
<h4>Задание 1</h4><br>
Датасет "MovieLens 20M Dataset" с рейтингами по фильмам https://grouplens.org/datasets/movielens/.
Какому фильму было выставлено больше всего оценок 5.0.

Файл-таблица с рейтингами:

In [3]:
ratings = pd.read_csv("C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Возьмем столбцы id и рейтинги, отфильтруем только оценки 5 в отдельный датафрейм:

In [4]:
ratings_5 = ratings[['movieId','rating']][ratings.rating == 5]

Посчитаем кол-во оценок для каждого id (по умолчанию будет сортировка по убыванию кол-ва):

In [5]:
ratings_5_count_series = ratings_5['movieId'].value_counts()

Посчитаем кол-во оценок для каждого id с помощью группировки, отсортируем по убыванию:

In [6]:
ratings_5_count = ratings_5.groupby('movieId').count().sort_values('rating', ascending = False)

##### Вариант 1

В ratings_5_count_series фильм с максимальным количеством рейтингов 5.0 находится в первой строке:

In [7]:
ratings_5_count_series.index[0]

318

В отсортированной группировке ratings_5_count фильм с максимальным количеством рейтингов 5.0 находится в первой строке:

In [8]:
ratings_5_count.head(1)

,rating
movieId,
318,170


Максимальное кол-во оценок 5 у фильма с id 318.

##### Вариант 2

Извлекаем id фильма с наибольшим кол-вом оценок 5:

In [9]:
ratings_5_count = ratings_5_count.reset_index()
max_ratings_5_count_mov_id = ratings_5_count.loc[0,'movieId']

Обращаемся к файлу-таблице с информацией о фильмах:

In [10]:
movies = pd.read_csv("C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Фильтруем информацию о фильмах по id с максимальным кол-вом 5.0 рейтингов и обращаемся к столбцу title:

In [11]:
movies[ movies.movieId == max_ratings_5_count_mov_id ]['title']

284    Shawshank Redemption, The (1994)
Name: title, dtype: object

Максимальное кол-во оценок 5 у фильма Побег из Шоушенка.

##### Вариант 3

Объединяем группировку с подсчетом оценок и информацию о фильмах, ответ будет в первой строке, так как группировка отсортирована:

In [12]:
joined = ratings_5_count.merge(movies, on = 'movieId', how = 'left')
joined.head(1)

,movieId,rating,title,genres
0,318,170,"Shawshank Redemption, The (1994)",Crime|Drama


И отдельно название:

In [13]:
joined.loc[0,'title']

'Shawshank Redemption, The (1994)'

##### Вариант 4

В объединении из Варианта 3 находим строку с максимальным кол-вом рейтингов:

In [14]:
joined[joined.rating == joined['rating'].max()]

,movieId,rating,title,genres
0,318,170,"Shawshank Redemption, The (1994)",Crime|Drama


<a href="#0"><h4>Наверх</h4></a>

<p id="2"> 
<h4>Задание 2</h4><br>
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [15]:
power_data = pd.read_csv("C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\power.csv")
power_data.head()

,country,year,quantity,category
0,Austria,1996,5.0,1
1,Austria,1995,17.0,1
2,Belgium,2014,0.0,1
3,Belgium,2013,0.0,1
4,Belgium,2012,35.0,1


Проверим написание стран Прибалтики. Они предположительно должны содержать комбинации букв: 'lith', 'est', 'lat':

In [16]:
print(power_data[ power_data['country'].str.contains('lith', case=False) ] ['country'].unique())
print(power_data[ power_data['country'].str.contains('est', case=False) ] ['country'].unique())
print(power_data[ power_data['country'].str.contains('lat', case=False) ] ['country'].unique())

['Lithuania']
['Estonia' 'State of Palestine' 'Timor-Leste']
['Latvia']


Отбираем данные по странам Балтики:

In [17]:
power_data_balt = power_data[ (power_data['country']=='Latvia') | \
                             (power_data['country']=='Lithuania') | \
                             (power_data['country']=='Estonia') ]

Отбираем данные за 2005-2010:

In [18]:
power_data_balt_05_10 = power_data_balt[ (power_data_balt['year'].between(2005, 2010)) ]

Отбираем категории 4, 12, 21:

In [19]:
power_data_balt_05_10_cat = power_data_balt_05_10[ (power_data_balt_05_10['category'].isin([4, 12, 21]))]

Суммируем значения, исключая неположительные:

In [20]:
power_data_balt_05_10_cat[ power_data_balt_05_10_cat['quantity'] > 0 ]['quantity'].sum()

240580.0

<a href="#0"><h4>Наверх</h4></a>

<p id="3">
<h4>Задание 3</h4><br>
Со страниц сайтов с табличными данными импортируйте таблицы в pandas dataframe.<br>
<h5>Часть 1</h5><br>
https://fortrader.org/quotes <br>
В коде этого сайта таблицы просто тегнуты table.

Сколько находит таблиц на странице:

In [21]:
len(pd.read_html('https://fortrader.org/quotes'))

9

Проверим, считывается ли без указания кодировки:

In [22]:
pd.read_html('https://fortrader.org/quotes')[0].head()

,Название,Цена,Изменение,Изменение(%),Обновлено
0,EURRUB Курс евро к рублю сегодня,87.29,-0.45,-0.51,NaN
1,USDRUB Курс рубля к доллару,77.06,-0.29,-0.37,NaN


Импортируем таблицы в соответствующие датафреймы:

In [23]:
eur_usd = pd.read_html('https://fortrader.org/quotes')[0]
raw = pd.read_html('https://fortrader.org/quotes')[1]
metal = pd.read_html('https://fortrader.org/quotes')[2]

In [24]:
eur_usd

,Название,Цена,Изменение,Изменение(%),Обновлено
0,EURRUB Курс евро к рублю сегодня,87.29,-0.45,-0.51,NaN
1,USDRUB Курс рубля к доллару,77.06,-0.29,-0.37,NaN


In [25]:
raw.head(2)

,Название,Цена,Изменение,Изменение(%),Обновлено
0,Нефть Brent Курс нефти сегодня,88.68,0.04,0.05,NaN
1,Нефть WTI Цены на нефть WTI,85.85,0.01,0.01,NaN


In [26]:
metal.head(2)

,Название,Цена,Изменение,Изменение(%),Обновлено
0,Золото Цена золота на бирже,1 838,6.00,0.00,NaN
1,Серебро Цена на серебро на бирже,24.16,-0.13,-0.51,NaN


##### Часть 2
https://www.finanz.ru/valyuty/v-realnom-vremeni <br>
На этом сайте таблицы помечены классами с названием.

Проверим, считывается ли без указания кодировки:

In [27]:
page_url_2 = 'https://www.finanz.ru/valyuty/v-realnom-vremeni'
pd.read_html(page_url_2)[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,аааа,3Т 439,"-2,2%",Nasdaq,13Т 769,"-2,7%",ааИбаКаОаЙаН,35Т 376,"-2,5%",USD/RUB,771960,"-0,5%"
1,а аЂаЁ,1Т 402,"-3,4%",S&P500,4Т 398,"-1,9%",ааЕббб,886,"0,8%",EUR/RUB,874160,"-0,7%"
2,Dow,34Т 265,"-1,3%",FTSE100,7Т 494,"-1,2%",ааОаЛаОбаО,1Т 836,"0,2%",EUR/USD,11331,"-0,1%"


Без кодировки не считывается, значит указываем ее при импорте:

In [28]:
pricebox = pd.read_html(page_url_2, encoding='utf-8')[0]
valute = pd.read_html(page_url_2, encoding='utf-8')[1]

In [29]:
pricebox.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11
0,ММВБ,3 439,"-2,2%",Nasdaq,13 769,"-2,7%",Биткойн,35 376,"-2,5%",USD/RUB,771960,"-0,5%"
1,РТС,1 402,"-3,4%",S&P500,4 398,"-1,9%",Нефть,886,"0,8%",EUR/RUB,874160,"-0,7%"


In [30]:
valute.head(2)

,Unnamed: 0,Рубль,Предложение,Спрос,Предыдущее закрытие,%,Абсолютное значение,Время
0,NaN,GBP/RUB,1046699,-,1051415,"-0,45%",-04716,07:12:00
1,NaN,EUR/RUB,874160,-,880150,"-0,68%",-05990,07:12:00


Первый столбец таблицы valute не считался, там картинки, уберем его:

In [31]:
valute = valute.drop(['Unnamed: 0'], axis = 1)
valute.head(1)

,Рубль,Предложение,Спрос,Предыдущее закрытие,%,Абсолютное значение,Время
0,GBP/RUB,1046699,-,1051415,"-0,45%",-04716,07:12:00


Альтернативно, так как таблицы помечены классами, к ним можно обратиться на названию:

In [32]:
pricebox_2 = pd.read_html(page_url_2, attrs = {'class': 'header_user_pricebox'}, encoding='utf-8')[0]
valute_2 = pd.read_html(page_url_2, attrs = {'class': 'quote_list'}, encoding='utf-8')[0]

In [33]:
pricebox_2.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11
0,ММВБ,3 439,"-2,2%",Nasdaq,13 769,"-2,7%",Биткойн,35 376,"-2,5%",USD/RUB,771960,"-0,5%"


In [34]:
valute_2.head(1)

,Unnamed: 0,Рубль,Предложение,Спрос,Предыдущее закрытие,%,Абсолютное значение,Время
0,NaN,GBP/RUB,1046699,-,1051415,"-0,45%",-04716,07:12:00


<a href="#0"><h4>Наверх</h4></a>

<p id="4">
<h4>Задание 4</h4><br>
Для датафрейма из файла log создайте столбец source_type по следующим правилам:<br>
* если источник traffic_source равен yandex или google, то в source_type ставится organic<br>
* для источников paid и email из России - ставим ad<br>
* для источников paid и email не из России - ставим other<br>
* все остальные варианты берем из traffic_source без изменений

In [35]:
log = pd.read_csv("C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\log.csv", sep=';')
log.head(2)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct


##### Вариант 1 (loc)

In [36]:
%%time

log.loc[log.traffic_source.isin(['yandex','google']),'source_type'] = 'organic'
log.loc[log.traffic_source.isin(['paid','email']) & (log.region == 'Russia'),'source_type'] = 'ad'
log.loc[log.traffic_source.isin(['paid','email']) & (log.region != 'Russia'),'source_type'] = 'other' 
log.loc[pd.isnull(log.source_type),['source_type']] = log.traffic_source

log.head()

Wall time: 35 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


##### Вариант 2 (функция)

In [37]:
def source_type(row):
    if row.traffic_source == 'yandex' or row.traffic_source == 'google':
        source_type = 'organic'
    elif (row.traffic_source == 'paid' or row.traffic_source == 'email') and row.region == 'Russia':
        source_type = 'ad'
    elif (row.traffic_source == 'paid' or row.traffic_source == 'email') and row.region != 'Russia':
        source_type = 'other'
    else:
        source_type = row.traffic_source        
    return source_type        

In [61]:
%%time

log['source_type'] = log.apply(source_type, axis=1)

log.head()

Wall time: 1.38 s


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


Функция отработала в 7 раз дольше, по скорости вариант с loc предпочтительнее.

<a href="#0"><h4>Наверх</h4></a>

<p id="5">
<h4>Задание 5</h4><br>
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. 
Прочитайте содержимое файла с датафрейм. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном.

In [38]:
url = pd.read_csv('C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\URLs.txt')
url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [39]:
url[url.url.str.contains('\/\d{8}\-', regex = True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


<a href="#0"><h4>Наверх</h4></a>

<p id="6">
<h4>Задание 6</h4><br>
Используйте файл с оценками фильмов ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [40]:
ratings = pd.read_csv('C:\\python\\portfolio\\data_analysis_python\\0_syntax\\data\\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Группируем датафрейм, для каждого id считаем кол-во оценок, минимум и максимум столбца timestamp:

In [41]:
user_grouped = ratings.groupby('userId').agg({'rating':'count','timestamp':{'min','max'}})
user_grouped.head()

rating   timestamp            
        count         min         max
userId                               
1          20  1260759108  1260759205
2          76   835355395   835356246
3          51  1298861589  1298932787
4         204   949778714   949982274
5         100  1163373044  1163375145

Добавляем столбец с расчетом времени между последним и первым рейтингами:

In [42]:
user_grouped['life'] = user_grouped['timestamp']['max'] - user_grouped['timestamp']['min']
user_grouped.head()

rating   timestamp                life
        count         min         max        
userId                                       
1          20  1260759108  1260759205      97
2          76   835355395   835356246     851
3          51  1298861589  1298932787   71198
4         204   949778714   949982274  203560
5         100  1163373044  1163375145    2101

Среднее время жизни пользователей с более 100 рейтингами, в днях:

In [43]:
round(user_grouped[user_grouped['rating']['count'] > 100]['life'].mean() / 60 / 60 / 24, 2)

463.89

<a href="#0"><h4>Наверх</h4></a>

<p id="7">
<h4>Задание 7</h4><br>
Дана статистика услуг перевозок клиентов компании по типам. Необходимо сформировать две таблицы:<br>
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента<br>
* аналогичную таблицу по типам выручки с указанием адреса клиента<br>
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [44]:
rzd = pd.DataFrame(
    {'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]})

auto = pd.DataFrame(
    {'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]})

air = pd.DataFrame(
    {'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]})

client_base = pd.DataFrame(
    {'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']})

Посчитаем суммы выручек по типам, чтобы потом проверить, что никакие данные не потерялись:

In [45]:
print('air ', air['air_revenue'].sum(),' auto ', auto['auto_revenue'].sum(),' rzd ', rzd['rzd_revenue'].sum())

air  271  auto  63410  rzd  20941


Проверим нет ли дубликатов id в таблицах, для последующего корректного объединения:

In [46]:
print('air ', len(air) == air['client_id'].nunique(),\
      ' auto ', len(auto) == auto['client_id'].nunique(),\
       ' rzd ', len(rzd) == rzd['client_id'].nunique())

air  True  auto  True  rzd  True


In [47]:
if any (air['client_id'].duplicated(keep = False).values): print('air duplicate')
if any (auto['client_id'].duplicated(keep = False).values): print('auto duplicate')
if any (rzd['client_id'].duplicated(keep = False).values): print('rzd duplicate') 

Объединяем три таблицы по столбцу client_id, оставляя все строки их всех таблиц:

In [48]:
all_revenue = (air.merge(auto, how = 'outer', on = 'client_id').merge(rzd, how = 'outer', on = 'client_id').
               sort_values('client_id').
               fillna(0))
all_revenue.head()

,client_id,air_revenue,auto_revenue,rzd_revenue
6,111,0.0,0.0,1093.0
7,112,0.0,0.0,2810.0
4,113,0.0,57483.0,10283.0
5,114,0.0,83.0,5774.0
0,115,81.0,912.0,981.0


Сверяем суммы выручек после объединения с суммами по отдельным таблицам, все сходится:

In [49]:
all_revenue.sum()

client_id         916.0
air_revenue       271.0
auto_revenue    63410.0
rzd_revenue     20941.0
dtype: float64

Добавляем к таблице all_revenue адреса клиентов:

In [50]:
all_revenue_add = all_revenue.merge(client_base, how = 'left', on = 'client_id')
all_revenue_add.head()

,client_id,air_revenue,auto_revenue,rzd_revenue,address
0,111,0.0,0.0,1093.0,Комсомольская 4
1,112,0.0,0.0,2810.0,Энтузиастов 8а
2,113,0.0,57483.0,10283.0,Левобережная 1а
3,114,0.0,83.0,5774.0,Мира 14
4,115,81.0,912.0,981.0,ЗЖБИиДК 1


<a href="#0"><h4>Наверх</h4></a>